<a href="https://colab.research.google.com/github/Camertronix-Cm/cbis-demos/blob/main/CBIS_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import numpy as np
import imageio as imio
import os
import torch
import tqdm
import cv2
import matplotlib.pyplot as plt
import random
import statistics
import copy
import math
import time
from google.colab.patches import cv2_imshow

xfeat = torch.hub.load('verlab/accelerated_features', 'XFeat', pretrained = True, top_k = 4096)
#Load some example images


Using cache found in /root/.cache/torch/hub/verlab_accelerated_features_main


In [75]:
def warp_corners_and_draw_matches(ref_points, dst_points, img1, img2):
    # Calculate the Homography matrix
    H, mask = cv2.findHomography(ref_points, dst_points, cv2.USAC_MAGSAC, 3.5, maxIters=1_000, confidence=0.999)
    mask = mask.flatten()

    # Get corners of the first image (image1)
    h, w = img1.shape[:2]
    corners_img1 = np.array([[0, 0], [w-1, 0], [w-1, h-1], [0, h-1]], dtype=np.float32).reshape(-1, 1, 2)

    # Warp corners to the second image (image2) space
    warped_corners = cv2.perspectiveTransform(corners_img1, H)

    # Draw the warped corners in image2
    img2_with_corners = img2.copy()
    for i in range(len(warped_corners)):
        start_point = tuple(warped_corners[i-1][0].astype(int))
        end_point = tuple(warped_corners[i][0].astype(int))
        cv2.line(img2_with_corners, start_point, end_point, (0, 255, 0), 4)  # Using solid green for corners

    # Prepare keypoints and matches for drawMatches function
    keypoints1 = [cv2.KeyPoint(p[0], p[1], 5) for p in ref_points]
    keypoints2 = [cv2.KeyPoint(p[0], p[1], 5) for p in dst_points]
    matches = [cv2.DMatch(i,i,0) for i in range(len(mask)) if mask[i]]
    #print(matches)

    # Draw inlier matches
    img_matches = cv2.drawMatches(img1, keypoints1, img2_with_corners, keypoints2, matches, None,
                                  matchColor=(0, 255, 0), flags=2)

    return img_matches


assert cv2.__version__[0] >= '3', 'The fisheye module requires opencv version >= 3.0.0'
def undistorted(img):
    K = np.array([[1462.891043541062, 0.0, 1058.282670166303], [0.0, 1471.5340747924379, 590.6436713184269],[0.0, 0.0, 1.0]])
    D = np.array([[-0.005693882560794027],
                  [-0.27694517294013893],
                  [0.4672123487246388],
                  [-0.3332243527055097]])

    '''K=np.array([[1446.4595928607794, 0.0, 1112.4044030939442], [0.0, 1454.7765408094683, 589.8928895307112], [0.0,
0.0, 1.0]])
    D=np.array([[0.06079819596228468], [-0.4708683686796481], [0.7183178914352084], [-0.5136644788920562]])'''
    Knew = K.copy()
    Knew[(0, 1), (0, 1)] = 0.4 * Knew[(0, 1), (0, 1)]
    img_undistorted = cv2.fisheye.undistortImage(img, K, D=np.array([0., 0., 0., 0.]), Knew=K)
    return img_undistorted

def blend_images(img1, img2, blend_width=20):

  height, width1, _ = img1.shape
  width2 = img2.shape[1]

        # Créez une nouvelle image pour stocker le résultat final
  result = np.zeros((height, width1 + width2 - blend_width, 3), dtype=img1.dtype)

        # Copiez la première image dans le résultat
  result[:, :width1 - blend_width, :] = img1[:, :width1 - blend_width, :]

        # Appliquez une transition douce sur la bande de mélange
  for i in range(blend_width):
    alpha = i / blend_width  # Fraction de mélange
    result[:, width1 - blend_width + i, :] = (1 - alpha) * img1[:, width1 - blend_width + i, :] + alpha * img2[:, i, :]

        # Copiez le reste de la deuxième image
  result[:, width1:, :] = img2[:, blend_width:, :]

  return result

def blend_images_vertical(img1, img2, blend_height=20):

  height1, width1, _ = img1.shape
  height2 = img2.shape[0]

  # Créez une nouvelle image pour stocker le résultat final
  result = np.zeros((height1 + height2 - blend_height, width1, 3), dtype=img1.dtype)

  # Copiez la première image dans le résultat
  result[:height1 - blend_height, :, :] = img1[:height1 - blend_height, :, :]

  # Appliquez une transition douce sur la bande de mélange
  for i in range(blend_height):
    alpha = i / blend_height  # Fraction de mélange
    result[height1 - blend_height + i, :, :] = (1 - alpha) * img1[height1 - blend_height + i, :, :] + alpha * img2[i, :, :]

  # Copiez le reste de la deuxième image
  result[height1:, :, :] = img2[blend_height:, :, :]

  return result

def adjust_brightness(image, current_brightness,target_brightness):
    factor = 0

    if current_brightness == 0:
        return image  # Évite la division par zéro
    else:

        factor = target_brightness / current_brightness
        if factor > 1.17:
            factor = 1.17

    adjusted_image = cv2.convertScaleAbs(image, alpha=factor, beta=0)
    return adjusted_image

def brightness_parameters (cropped_img1, cropped_img2):

    current_brightness1= np.mean(cropped_img1)
    current_brightness2 = np.mean(cropped_img2)
    current_brightness = (current_brightness1 +current_brightness2)/2

    difference = current_brightness1-current_brightness2

    # Vérification que la différence est comprise entre -16 et 16
    if -16 <= difference <= 16:
        # On extrait la partie entière et les décimales
        entier = int(difference)  # Partie entière
        decimal = abs(difference - entier)  # Partie décimale

        # Conversion de la partie décimale en chaîne pour vérifier les chiffres
        decimal_str = str(decimal)[2:]  # On enlève '0.'

        # Gestion de la majoration ou minoration
        if len(decimal_str) > 1 and int(decimal_str[1]) >= 5:
            # Si le deuxième chiffre est supérieur ou égal à 5, on arrondit
            valeur_finale = entier + 1
        else:
            # Sinon, on garde la partie entière
            valeur_finale = entier

        # Calcul de target_brightness
        if difference < 0:
            brightness_correction = valeur_finale * (-2)
        elif difference > 0:
            brightness_correction = valeur_finale * (-2)
        else:  # difference == 0
            brightness_correction = 0
    else:
        brightness_correction = 0  # Si la différence n'est pas dans la plage

    if current_brightness1 <current_brightness2:

            target_brightness = current_brightness2
            if target_brightness/current_brightness < 1:
                cropped_img1 = adjust_brightness(cropped_img1, current_brightness,target_brightness+brightness_correction)

            if target_brightness/current_brightness> 1:
                cropped_img2 = adjust_brightness(cropped_img2, current_brightness,target_brightness+brightness_correction)

    elif current_brightness1 >current_brightness2:

            target_brightness = current_brightness1
            if target_brightness/current_brightness < 1:
                cropped_img2 = adjust_brightness(cropped_img2, current_brightness,target_brightness+brightness_correction)

            if target_brightness/current_brightness> 1:
                cropped_img1 = adjust_brightness(cropped_img1, current_brightness,target_brightness+brightness_correction)


    return cropped_img1, cropped_img2


def pad_imagewidth(image, target_width,side):
  height, width = image.shape[:2]
  if side=='r':
    padding_left=0
    padding_right = target_width - width
  elif side=='l':
    padding_right=0
    padding_left = target_width - width

  # Pad the image with black borders on left and right
  padded_image = cv2.copyMakeBorder(image, 0, 0, padding_left, padding_right, cv2.BORDER_CONSTANT, value=(0, 0, 0))

  return padded_image

def image_translation(img1, img2, shift_y1,repetitions):

    height, width = img1.shape[:2]
    height2, width2 = img2.shape[:2]

    if shift_y1 > 0:
        # TRAITEMENT SUR LA PREMIÈRE IMAGE
        last_line = img1[-1:, :, :]
        repeat_line = np.tile(last_line, (repetitions, 1, 1))
        new_height1 = img1.shape[0] + repetitions
        img_resultat = np.zeros((new_height1, img1.shape[1], img1.shape[2]), dtype=img1.dtype)

        img_resultat[:img1.shape[0], :, :] = img1
        img_resultat[img1.shape[0]:, :, :] = repeat_line
        img1 = img_resultat

        # TRAITEMENT SUR LA DEUXIÈME IMAGE
        first_line = img2[0:1, :, :]
        repeat_line2 = np.tile(first_line, (repetitions, 1, 1))
        new_height2 = img2.shape[0] + repetitions
        img_resultat2 = np.zeros((new_height2, img2.shape[1], img2.shape[2]), dtype=img2.dtype)

        img_resultat2[:repetitions, :, :] = repeat_line2
        img_resultat2[repetitions:, :, :] = img2
        img2 = img_resultat2
    else:
        # TRAITEMENT SUR LA PREMIÈRE IMAGE
        last_line = img2[-1:, :, :]
        repeat_line2 = np.tile(last_line, (repetitions, 1, 1))
        new_height2 = img2.shape[0] + repetitions
        img_resultat2 = np.zeros((new_height2, img2.shape[1], img2.shape[2]), dtype=img2.dtype)

        img_resultat2[:img2.shape[0], :, :] = img2
        img_resultat2[img2.shape[0]:, :, :] = repeat_line2
        img2 = img_resultat2

        # TRAITEMENT SUR LA DEUXIÈME IMAGE
        first_line = img1[0:1, :, :]
        repeat_line = np.tile(first_line, (repetitions, 1, 1))
        new_height1 = img1.shape[0] + repetitions
        img_resultat = np.zeros((new_height1, img1.shape[1], img1.shape[2]), dtype=img1.dtype)

        img_resultat[:repetitions, :, :] = repeat_line
        img_resultat[repetitions:, :, :] = img1
        img1 = img_resultat

    # Retourner les images modifiées
    return img1, img2

def translate_and_concatenate(img1, img2, dy):

  h1, w1 = img1.shape[:2]
  h2, w2 = img2.shape[:2]

  # Calculate the new height of the translated image
  new_h2 = h2 + abs(dy)

  # Create a new image with the calculated height
  translated_img2 = np.zeros((new_h2, w2, 3), dtype=img2.dtype)
  pad_img1=np.zeros((new_h2, w1, 3), dtype=img1.dtype)

  # Copy the original image to the new image, applying the translation
  if dy >= 0:
    translated_img2[dy:, :] = img2
    pad_img1[:new_h2-dy, :] = img1
  else:
    translated_img2[:new_h2+dy, :] = img2
    pad_img1[abs(dy):, :] = img1

  concatenated_img=blend_images(pad_img1,translated_img2,20)

  return concatenated_img

def image_stitching(im1, im2):
  im1_undistorted=undistorted(im1)
  im2_undistorted=undistorted(im2)
  debut=time.time()
  #im1_undistorted=im1
  #im2_undistorted=im2
  mkpts_0, mkpts_1 = xfeat.match_xfeat(im1_undistorted, im2_undistorted, top_k = 4096, min_cossim=-1 )
  canvas = warp_corners_and_draw_matches(mkpts_0, mkpts_1, im1_undistorted, im2_undistorted)
  category1, category2, category3, category4, category5, category6, category7,category8,category9,category10,category11,category12,category13,category14,category15,category16,category17,category18,category19,category20, summoy, nbre =0,0,0,0,0,0,0, 0, 0,0,0,0,0,0,0,0,0,0,0,0,0,0
  yaxis_class=[[0,100],[100,200],[200,300],[300,400],[400,500],[500,600],[600,700],[700,800],[800,900],[900,1000],[1000,1080]]
  xaxis_class=[[0,100],[100,200],[200,300],[300,400],[400,500],[500,600],[600,700],[700,800],[800,900],[900,950],[950,1000],[1000,1100],[1100,1200],[1200,1300],[1300,1400],[1400,1500],[1500,1600],[1600,1700],[1700,1800],[1800,1900],[1900,2000]]
  sum_result=None
  fixed_point=[]
  average_xaxis=[]
  average_yaxis=[]
  filtered_setleft,filtered_setright=[],[]
  k=0
  i,avg_abs,avg_ord1, avg_ord2,ydiff,bad_frame=0,0,0,0,0,0
  area,ydiff_left=[],[]
  concatpoint=None
  #im1_undistorted,im2_undistorted=brightness_parameters(im1_undistorted,im2_undistorted)

  yleft_right,yright_left=0,0
  for y in range(len(mkpts_0)):
    if mkpts_0[y][0]>mkpts_1[y][0]:
      if mkpts_0[y][1]>mkpts_1[y][1]:
        filtered_setleft.append([mkpts_0[y],mkpts_1[y]])
        ydiff_left.append(mkpts_0[y][1]-mkpts_1[y][1])
        yleft_right+=1
      elif mkpts_0[y][1]<mkpts_1[y][1]:
        filtered_setright.append([mkpts_0[y],mkpts_1[y]])
        ydiff+=mkpts_1[y][1]-mkpts_0[y][1]
        yright_left+=1
  ydiff=ydiff/yright_left
  ydiff_left=statistics.mean(ydiff_left)

  filtered_setleft=[m for m in filtered_setleft if m[0][1]-m[1][1]<ydiff_left]
  filtered_setright=[m for m in filtered_setright if  m[1][1]-m[0][1]<ydiff]


  if yleft_right>yright_left:

    for elt in xaxis_class:
      xdiff=[]

      summoy1,summoy,nbre=0,0,0
      for m,n in filtered_setleft:
        if m[0]>= elt[0] and m[0]<elt[1] :
          xdiff.append(m[0]-n[0])
      try:
        avgxdiff=statistics.mean(xdiff)
        average_xaxis.append([avgxdiff,len(xdiff)])
      except statistics.StatisticsError:
        avgxdiff=0
        average_xaxis.append([avgxdiff,len(xdiff)])

    gen_avg=statistics.mean([sublist[0] for sublist in average_xaxis])

    diff_avg=[]
    for j in range(len(average_xaxis)-1):
      if average_xaxis[j][1]!=0 and average_xaxis[j+1][1]!=0:
        diff_avg.append([abs(average_xaxis[j][0] -average_xaxis[j+1][0]),j])

    small_diff,succ=[], 10
    while len(small_diff)==0:
      #small_diff=[num for num in diff_avg[6:16] if num[0]<=succ and average_xaxis[num[1]][1]!=0]
      small_diff=[num for num in diff_avg if num[0]<=succ and average_xaxis[num[1]][1]!=0]
      succ+=5

    #copysmall_diff=copy.deepcopy(small_diff)
    result = []
    current_group = []
    for element in small_diff:
      if not current_group or abs(element[1] - current_group[-1][1]) <= 1:
        current_group.append(element)
      else:
        result.append(current_group)
        current_group = [element]
    if current_group:
      result.append(current_group)

    diff_genavg=[]
    area= [[] for _ in range(len(result))]
    for e in range(len(result)):

      area[e].append(xaxis_class[result[e][0][1]][0])
      area[e].append(xaxis_class[result[e][-1][1]+1][1])

    diff_genavg=[]
    if len(area)>1:
      filtered_set=[[] for _ in range(len(result))]
      for i in range (len(area)):

        data=[m[0]-n[0] for m,n in filtered_setleft if area[i][0] <= m[0] <= area[i][1]]
        n = len(data)
        class_avg = statistics.mean(data)
        var= sum((x - class_avg) ** 2 for x in data) / n
        ecart=math.sqrt(var)
        diff_genavg.append([area[i],abs(class_avg-gen_avg)/n,[class_avg,ecart]])

        diff_genavg=sorted(diff_genavg, key=lambda x:x[1] )

      area=[m[0] for m in diff_genavg ]

      filtered_setcopy=[]
      for i in range (len(area)):

        error=diff_genavg[i][-1][1]/2
        class_avg=diff_genavg[i][-1][0]

        while len(filtered_set[i])==0 and error<diff_genavg[i][-1][1]/2+10:


          if class_avg<gen_avg+15:
            filtered_set[i]=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setleft if area[i][0] <= m[0] <= area[i][1] and class_avg <m[0]-n[0]<class_avg+error]
          else:
            filtered_set[i]=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setleft if area[i][0] <= m[0] <= area[i][1] and class_avg-error <m[0]-n[0]<class_avg+error]
          error+=5

        if len(filtered_set[i])>0:
          filtered_set[i]=sorted(filtered_set[i], key=lambda x:x[-1] )
          filtered_set[i]=[elt[:2] for elt in filtered_set[i]]
          filtered_setcopy.append(filtered_set[i])
      #print(len(filtered_setcopy),filtered_setcopy)

      try:

        concatpoint=filtered_setcopy[0][int(len(filtered_setcopy)/2)][:2]

      except IndexError:
        print("pas de points")

    else:
      filtered_setleft=[[m,n] for m,n in filtered_setleft if area[0][0] <= m[0] <= area[0][1]]
      try:
        concatpoint=filtered_setleft[int(len(filtered_setleft)/2)][:2]

      except IndexError:
        print("pas de points")

  else:

    for elt in xaxis_class:
      xdiff=[]

      summoy1,summoy,nbre=0,0,0
      for m,n in filtered_setright:
        if elt[0]<= m[0]<elt[1] :

          xdiff.append(m[0]-n[0])
      try:
        avgxdiff=statistics.mean(xdiff)
        average_xaxis.append([avgxdiff,len(xdiff)])
      except statistics.StatisticsError:
        avgxdiff=0
        average_xaxis.append([avgxdiff,len(xdiff)])
    gen_avg=statistics.mean([sublist[0] for sublist in average_xaxis])

    diff_avg=[]
    for j in range(len(average_xaxis)-1):
      if average_xaxis[j][1]!=0 and average_xaxis[j+1][1]!=0:
        diff_avg.append([abs(average_xaxis[j][0] -average_xaxis[j+1][0]),j])
      #diff_avg.append([abs(average_xaxis[j][0] -average_xaxis[j+1][0]),j])

    small_diff,succ=[], 10

    while len(small_diff)==0:
      #small_diff=[num for num in diff_avg[6:16] if num[0]<=succ and average_xaxis[num[1]][1]!=0]
      small_diff=[num for num in diff_avg if num[0]<=succ and average_xaxis[num[1]][1]!=0]
      succ+=5

    copysmall_diff=copy.deepcopy(small_diff)
    result = []
    current_group = []
    for element in small_diff:
      if not current_group or abs(element[1] - current_group[-1][1]) <= 1:
        current_group.append(element)
      else:
        result.append(current_group)
        current_group = [element]
    if current_group:
      result.append(current_group)

    area= [[] for _ in range(len(result))]
    #area= [[] for _ in diff_avg]
    diff_genavg=[]

    for e in range(len(result)):

      area[e].append(xaxis_class[result[e][0][1]][0])
      area[e].append(xaxis_class[result[e][-1][1]+1][1])

    diff_genavg=[]
    if len(area)>1:
      filtered_set=[[] for _ in range(len(result))]

      for i in range (len(area)):

        data=[m[0]-n[0] for m,n in filtered_setright if area[i][0] <= m[0] <= area[i][1]]
        n = len(data)
        class_avg = statistics.mean(data)
        var= sum((x - class_avg) ** 2 for x in data) / n
        ecart=math.sqrt(var)
        diff_genavg.append([area[i],abs(class_avg-gen_avg)/n,[class_avg,ecart]])

        diff_genavg=sorted(diff_genavg, key=lambda x:x[1] )
      #print(diff_genavg)

      area=[m[0] for m in diff_genavg ]
      #print('area',area)
      filtered_setcopy=[]
      for i in range (len(area)):

        error=diff_genavg[i][-1][1]/2
        class_avg=diff_genavg[i][-1][0]

        while len(filtered_set[i])==0 and error<diff_genavg[i][-1][1]/2+10:

          #print("class_avg,error,area",class_avg,error,area[i])

          if class_avg<gen_avg+15:
            filtered_set[i]=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setright if area[i][0] <= m[0] <= area[i][1] and class_avg < m[0]-n[0]<class_avg+error]
          else:
            filtered_set[i]=[[m,n,m[0]-n[0]-class_avg] for m,n in filtered_setright if area[i][0] <= m[0] <= area[i][1] and class_avg-error <m[0]-n[0]<class_avg+error]
          error+=5

        if len(filtered_set[i])>0:
          filtered_set[i]=sorted(filtered_set[i], key=lambda x:x[-1] )
          filtered_set[i]=[elt[:2] for elt in filtered_set[i]]
          filtered_setcopy.append(filtered_set[i])
      #print(len(filtered_setcopy),filtered_setcopy)

      try:

        concatpoint=filtered_setcopy[0][int(len(filtered_setcopy)/2)][:2]

      except IndexError:
        print("pas de points")

    else:
      filtered_setright=[[m,n] for m,n in filtered_setright if area[0][0] <= m[0] <= area[0][1]]
      try:
        concatpoint=filtered_setright[int(len(filtered_setright)/2)][:2]

      except IndexError:
        print("pas de points")


  if concatpoint!=None:


    #print(concatpoint)
    x1,y1=concatpoint[0]
    x2,y2=concatpoint[1]
    shift_y=math.ceil(y1-y2)
    cropped_img1 = im1_undistorted[0:im1_undistorted.shape[0],0:int(x1)+15]
    cropped_img2 = im2_undistorted[0:im2_undistorted.shape[0],int(x2)-15:im2_undistorted.shape[1]]
    cropped_img1,cropped_img2=brightness_parameters(cropped_img1,cropped_img2)
    result_horizontal=translate_and_concatenate(cropped_img1,cropped_img2,shift_y)

  else:
    result_horizontal=None
  fin=time.time()

  return sum_result,concatpoint

def reduced_concatenation(img1,img2, fixed_point) :
  yaxis_class=[[0,100],[100,200],[200,300],[300,400],[400,500],[500,600],[600,700],[700,800],[800,900],[900,1000],[1000,1080]]
  print("fixed_point",len(fixed_point))

  im1_undistorted=undistorted(img1)
  im2_undistorted=undistorted(img2)
  #im1_undistorted=img1
  #im2_undistorted=img2
  x1, y1 = fixed_point[0]
  x2, y2 = fixed_point[1]
  cropped_img1 = im1_undistorted[0:im1_undistorted.shape[0],0:int(x1)+15]
  cropped_img2 = im2_undistorted[0:im2_undistorted.shape[0],int(x2)-15:im2_undistorted.shape[1]]
  cropped_img1,cropped_img2=brightness_parameters(cropped_img1,cropped_img2)
  shift_y=math.ceil(y1-y2)
  sum_result=translate_and_concatenate(cropped_img1,cropped_img2,shift_y)
  return sum_result


In [80]:
def capture_and_concatenate(video_directory, output_file):
    # Charger les vidéos depuis le répertoire
    video_files = [os.path.join(video_directory, f) for f in os.listdir(video_directory) if f.endswith('.mp4')]
    video_files.sort()
    print(video_files)

    if len(video_files) < 2:
        print("Erreur : Au moins deux fichiers vidéo sont nécessaires pour la concaténation.")
        return

    # Obtenez les propriétés des deux premières vidéos
    def get_video_properties(video_path):
        cap = cv2.VideoCapture(video_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        return (width, height), fps, total_frames

    (width1, height1), fps1, total_frames1 = get_video_properties(video_files[0])
    (width2, height2), fps2, total_frames2 = get_video_properties(video_files[1])

    # Utiliser la hauteur minimale et additionner les largeurs pour la concaténation
    frame_height = min(height1, height2)
    #fps = min(fps1, fps2)
    fps=5

    print(f"Dimensions initiales : Hauteur = {frame_height}, FPS = {fps}",total_frames1,total_frames2,fps1,fps2)

    # Initialiser les captureurs vidéo pour les deux vidéos
    caps = [cv2.VideoCapture(video_file) for video_file in video_files[:2]]

    # Définir le codec et créer un objet VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None  # On initie 'out' ici, il sera défini plus tard
    refresh,i,fixed_points=0,0,[]

    while True:
        frames = []
        reached_end = False

        # Lire les frames des deux vidéos
        for cap in caps:
            grabbed, frame = cap.read()
            if grabbed:
                # Redimensionner les frames à la hauteur minimale tout en conservant le ratio d'aspect
                #frame = cv2.resize(frame, (int(frame.shape[1] * (frame_height / frame.shape[0])), frame_height))
                frames.append(frame)

            else:
                reached_end = True

        if reached_end:
            print("Fin de l'une des vidéos. Arrêt du processus.")
            break

        # Vérification que nous avons bien deux frames valides
        if len(frames) == 2:

          essai=False
          if refresh % 6 == 0:
            result_frame,fixed_point=image_stitching(frames[0], frames[1])
            if result_frame is None:
              if len(fixed_points)>0:
                result_frame=reduced_concatenation(frames[0], frames[1],fixed_points[-1])

            else:
              fixed_points.append(fixed_point)
          else:
            result_frame=reduced_concatenation(frames[0], frames[1],fixed_point)

          print("refresh" ,refresh)
          #cv2.imwrite("/content/sample_data/video_from_images/result_frame"+str(refresh)+".jpg", result_frame)
          refresh+=1


          if result_frame is not None and result_frame.shape[0] > 0 and result_frame.shape[1] > 0:
            print(f"Frame concaténée écrite avec succès. Dimensions : {result_frame.shape}")

            frame_width = 2500
            #frame_height = result_frame.shape[0]
            frame_height=1100

            if out is None:  # Initialiser out la première fois

              out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc('X', 'V', 'I', 'D'), fps, (frame_width, frame_height))

            print("result_frame.shape", result_frame.shape)
            result_frame = cv2.resize(result_frame, (frame_width, frame_height))
            out.write(result_frame)  # Écrire la frame concaténée dans la vidéo de sortie


        else:
          print("Erreur : Moins de deux frames disponibles pour la concaténation.")

    for cap in caps:
        cap.release()
    if out is not None:
      out.release()
    #cv2.destroyAllWindows()

# Exemple d'utilisation
capture_and_concatenate('/content/sample_data/videos', 'output_video1.mp4')


['/content/sample_data/videos/camera0.mp4', '/content/sample_data/videos/camera1.mp4']
Dimensions initiales : Hauteur = 1080, FPS = 5 102 101 5.0 5.0
refresh 0
fixed_point 2
refresh 1
Frame concaténée écrite avec succès. Dimensions : (1088, 2391, 3)
result_frame.shape (1088, 2391, 3)
fixed_point 2
refresh 2
Frame concaténée écrite avec succès. Dimensions : (1088, 2391, 3)
result_frame.shape (1088, 2391, 3)
fixed_point 2
refresh 3
Frame concaténée écrite avec succès. Dimensions : (1088, 2391, 3)
result_frame.shape (1088, 2391, 3)
fixed_point 2
refresh 4
Frame concaténée écrite avec succès. Dimensions : (1088, 2391, 3)
result_frame.shape (1088, 2391, 3)
fixed_point 2
refresh 5
Frame concaténée écrite avec succès. Dimensions : (1088, 2391, 3)
result_frame.shape (1088, 2391, 3)
refresh 6
fixed_point 2
refresh 7
Frame concaténée écrite avec succès. Dimensions : (1090, 2374, 3)
result_frame.shape (1090, 2374, 3)
fixed_point 2
refresh 8
Frame concaténée écrite avec succès. Dimensions : (1090,

In [77]:
cap = cv2.VideoCapture("output_video1.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
cap.release()
print(f"Dimensions initiales : Hauteur = {height}, FPS = {fps}",total_frames)

Dimensions initiales : Hauteur = 1100, FPS = 5.0 86
